In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [31]:
pd.set_option('display.max_columns', 100)

In [32]:
pickle_path = '/Users/jacoblucas/group_project/Baseball/run/setup/data/analysis_dataset.pickle'
df_odds = pd.read_pickle(pickle_path)
df_allbets = pd.read_pickle('portfolio.pickle')

In [33]:
df = pd.merge(df_odds, df_allbets, on='gameno', how='right')

In [34]:
df.head()

,date,gameno,visitor,v_team,v_team_run_total,v_money_line_close,v_money_line_open,v_run_line_close,v_run_line_odds_close,v_pitcher,v_rot,v_run_dif_game,home,h_team,h_team_run_total,h_money_line_close,h_money_line_open,h_run_line_close,h_run_line_odds_close,h_pitcher,h_rot,h_run_dif_game,total_runs_game,over_line_close,over_line_open,over_odds_close,over_odds_open,under_line_close,under_line_open,under_odds_close,under_odds_open,Bet_Outcomes,Date,Portfolio_Value,home_bet_outcomes_ml,home_portfolio_value_ml,visitor_bet_outcomes_ml,visitor_portfolio_value_ml,favorites_bet_outcomes_ml,favorites_portfolio_value_ml,underdogs_bet_outcomes_ml,underdogs_portfolio_value_ml,home_underdogs_ml_bet_outcomes_ml,home_underdogs_ml_portfolio_value_ml,visitor_favorites_ml_bet_outcomes_ml,visitor_favorites_ml_portfolio_value_ml,visitor_underdogs_ml_bet_outcomes_ml,visitor_underdogs_ml_portfolio_value_ml,home_favorites_ml_bet_outcomes_ml,home_favorites_ml_portfolio_value_ml,home_bet_outcomes_rl,home_portfolio_value_rl,visitor_bet_outcomes_rl,visitor_portfolio_value_rl,favorites_bet_outcomes_rl,favorites_portfolio_value_rl,underdogs_bet_outcomes_rl,underdogs_portfolio_value_rl,home_favorites_rl_bet_outcomes_rl,home_favorites_rl_portfolio_value_rl,visitor_underdogs_rl_bet_outcomes_rl,visitor_underdogs_rl_portfolio_value_rl,home_underdogs_rl_bet_outcomes_rl,home_underdogs_rl_portfolio_value_rl,visitor_favorites_rl_bet_outcomes_rl,visitor_favorites_rl_portfolio_value_rl,overs_bet_outcomes_ou,overs_portfolio_value_ou,unders_bet_outcomes_ou,unders_portfolio_value_ou,favorites_bet_outcomes_ou,favorites_portfolio_value_ou,underdogs_bet_outcomes_ou,underdogs_portfolio_value_ou
0,1.270440e+09,1,V,PHI,11.0,-200.0,-200.0,NaN,NaN,RHALLADAY-R,901.0,10.0,H,WAS,1.0,175.0,175.0,NaN,NaN,JLANNAN-L,902.0,-10.0,12.0,7.5,7.5,-125.0,100.0,7.5,7.5,105.0,-120.0,0.500000,1270440000.0,50.000000,-1.000000,-14.529915,0.500000,-50.00000,0.500000,135.470085,-1.00,-200.0,-1.00,-100.0,0.500000,50.00000,0.00,-100.0,0.000000,85.470085,-1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.800000,185.000000,-1.0,-200.0,0.800000,-20.000000,-1.00,5.0
1,1.270440e+09,2,V,MIA,1.0,-102.0,105.0,NaN,NaN,JJOHNSON-R,903.0,-6.0,H,NYM,7.0,-108.0,-115.0,NaN,NaN,JOSANTANA-L,904.0,6.0,8.0,7.0,7.0,-130.0,-125.0,7.0,7.0,110.0,105.0,0.925926,1270440000.0,142.592593,0.925926,78.062678,-1.000000,-150.00000,0.925926,228.062678,-1.00,-300.0,0.00,-100.0,0.000000,50.00000,-1.00,-200.0,0.925926,178.062678,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,NaN,0.0,0.0,0.0,0.0,0.769231,261.923077,-1.0,-300.0,0.769231,56.923077,-1.00,-95.0
2,1.270440e+09,3,V,STL,11.0,-143.0,-155.0,NaN,NaN,CARPENTER-R,905.0,5.0,H,CIN,6.0,128.0,140.0,NaN,NaN,AHARANG-R,906.0,-5.0,17.0,7.5,7.5,-105.0,-105.0,7.5,7.5,-115.0,-115.0,0.699301,1270440000.0,212.522663,-1.000000,-21.937322,0.699301,-80.06993,0.699301,297.992748,-1.00,-400.0,-1.00,-200.0,0.699301,119.93007,0.00,-200.0,0.000000,178.062678,-1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.050000,366.923077,-1.0,-400.0,-1.000000,-43.076923,1.05,10.0
3,1.270440e+09,4,V,LAD,5.0,-126.0,-155.0,NaN,NaN,VPADILLA-R,907.0,-6.0,H,PIT,11.0,111.0,140.0,NaN,NaN,ZDUKE-L,908.0,6.0,16.0,8.5,8.5,-120.0,-110.0,8.5,8.5,100.0,-110.0,-1.000000,1270440000.0,112.522663,1.110000,89.062678,-1.000000,-180.06993,-1.000000,197.992748,1.11,-289.0,1.11,-89.0,-1.000000,19.93007,0.00,-200.0,0.000000,178.062678,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,NaN,0.0,0.0,0.0,0.0,0.833333,450.256410,-1.0,-500.0,0.833333,40.256410,-1.00,-90.0
4,1.270440e+09,5,V,COL,5.0,-105.0,110.0,NaN,NaN,UJIMENEZ-R,909.0,2.0,H,MIL,3.0,-105.0,-125.0,NaN,NaN,YGALLARDO-R,910.0,-2.0,8.0,7.5,7.5,105.0,-105.0,7.5,7.5,-125.0,-115.0,-1.000000,1270440000.0,12.522663,-1.000000,-10.937322,1.050000,-75.06993,-1.000000,97.992748,1.05,-184.0,0.00,-89.0,0.000000,19.93007,1.05,-95.0,-1.000000,78.062678,-1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.050000,555.256410,-1.0,-600.0,-1.000000,-59

In [61]:
v_index_ls = []
h_index_ls = []
for index, row in df.iterrows():
    v_ml_open = row['v_money_line_open']
    v_ml_close = row['v_money_line_close']
    h_ml_open = row['h_money_line_open']
    h_ml_close = row['h_money_line_close']
    
    if v_ml_close < v_ml_open:
        v_index_ls.append(index)
    
    if h_ml_close < h_ml_open:
        h_index_ls.append(index)
v_pub_fav = df.iloc[v_index_ls, :]
h_pub_fav = df.iloc[h_index_ls, :]

In [65]:
# bet on public sentiment underdogs
home_pub_dog_winners = v_pub_fav.loc[vis_pub_fav['h_run_dif_game'] > 0]
hpd_percent_win = len(home_pub_dog_winners) / len(v_pub_fav)
hpd_profit = v_pub_fav['home_underdogs_ml_bet_outcomes_ml'].sum()

vis_pub_dog_winners = h_pub_fav.loc[h_pub_fav['v_run_dif_game'] > 0]
vpd_percent_win = len(vis_pub_dog_winners) / len(h_pub_fav)
vpd_profit = h_pub_fav['visitor_underdogs_ml_bet_outcomes_ml'].sum()

overall_percent_win = (hpd_percent_win + vpd_percent_win) / 2
overall_profit = hpd_profit + vpd_profit
overall_percent_win, overall_profit

(0.4711432916533334, -534.3349999999999)

In [35]:
# create dfs based on different metrics
heavy_home_fav = df.loc[(df['h_money_line_close'] < (-200))]
heavy_vis_fav = df.loc[(df['v_money_line_close'] < (-200))]
home_fav = df[df['h_money_line_close'].between(-200, 0, inclusive=True)]
vis_fav = df[df['v_money_line_close'].between(-200, 0, inclusive=True)]

In [41]:
# profitability of heavy home favorites
heavy_home_fav_winners = heavy_home_fav.loc[heavy_home_fav['h_run_dif_game'] > 0]
h_percent_win = len(heavy_home_fav_winners) / len(heavy_home_fav)
profit = heavy_home_fav['home_favorites_ml_bet_outcomes_ml'].sum()
h_percent_win, profit

(0.7047461368653422, -2.3076273651722605)

In [42]:
# profitability of normal home favorites
home_fav_winners = home_fav.loc[home_fav['h_run_dif_game'] > 0]
h_percent_win = len(home_fav_winners) / len(home_fav)
profit = home_fav['home_favorites_ml_bet_outcomes_ml'].sum()
h_percent_win, profit

(0.5614160814103483, -383.8112701050886)

In [43]:
# profitability of heavy vis favorites
heavy_vis_fav_winners = heavy_vis_fav.loc[heavy_vis_fav['v_run_dif_game'] > 0]
v_percent_win = len(heavy_vis_fav_winners) / len(heavy_vis_fav)
profit = heavy_vis_fav['visitor_favorites_ml_bet_outcomes_ml'].sum()
v_percent_win, profit

(0.6750700280112045, -12.632557564125193)

In [44]:
# profitability of normal vis favorites
vis_fav_winners = vis_fav.loc[vis_fav['v_run_dif_game'] > 0]
v_percent_win = len(vis_fav_winners) / len(vis_fav)
profit = vis_fav['visitor_favorites_ml_bet_outcomes_ml'].sum()
v_percent_win, profit

(0.5452621093263603, -159.97104644470477)